### Pipeline for QnA with Memory

In [61]:
####Notes####
# chunked data length can be atmost 16 for azureopenai

In [62]:
# !pip3 install langchain
# !pip install openai
# !pip3 install openai chromadb
# !pip3 install tiktoken

In [63]:
# !pip3 install unstructured
# !pip3 install pdf2image
# !pip3 install pdfminer
# !pip3 install pdfminer.six
# !pip3 install pymupdf 

In [64]:
import os
import sys
import glob
import re
import importlib
import langchain
from langchain.document_loaders import WebBaseLoader, UnstructuredPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate
sys.path.append(os.path.join(os.getcwd(), '../scripts'))
import qna_memory_jn as qna_m
import constants as c
importlib.reload(qna_m)
importlib.reload(c)

<module 'constants' from '/Users/rachitjoshi/Documents/Finbot/notebooks/../scripts/constants.py'>

#### Environment

In [65]:
os.environ["OPENAI_API_KEY"] = "6cdb659e5a9d402e80c212fe8ea26483"

In [66]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://test-chatgpt-flomoney.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

#### Parameters

In [67]:
save_folder = '../data/pdfs/'
qna_prompt_template = os.path.join(os.getcwd(), '../scripts/qna_prompt_template.txt')
# prompt_template_file = None
condense_question_template = os.path.join(os.getcwd(), '../scripts/condense_question_template.txt')
# condense_question_template = None
pdf_list = [
    _ for _ in glob.glob(os.path.join(os.getcwd(), save_folder, '*.pdf'))
]
web_list = []

#### QnA Chain 

In [68]:
langchain_qna = qna_m.LangchainQnA(c.chunking_interface, c.embedding_model)

In [69]:
qna_chain = langchain_qna.main_function(
    pdf_list, web_list, qna_prompt_template, condense_question_template
)

#### QnA (using Chain)

In [72]:
question = 'What are the advantages of'
result = qna_chain({"question": question})

[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "question": "Does Nvidia paln to make cars",
  "chat_history": "\nHuman: Ok thank you\nAssistant:  You're welcome. Have a great day!\n<|im_end|>"
}
[llm/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain > 3:llm:AzureOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n\nHuman: Ok thank you\nAssistant:  You're welcome. Have a great day!\n<|im_end|>\nFollow Up Input: Does Nvidia paln to make cars\nStandalone question:"
  ]
}
[llm/end] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain > 3:llm:AzureOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "<|im_end|>",
    

[llm/end] [1:chain:ConversationalRetrievalChain > 5:chain:StuffDocumentsChain > 6:chain:LLMChain > 7:llm:AzureOpenAI] [3.07s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " The article discusses the BlackRock Investment Institute's new playbook, which leads them to favor emerging market (EM) over developed market (DM) assets. They believe that EM assets have an edge over DM peers in the first layer of their new playbook, the macro assessment. Inflation is cooling enough in key EMs to allow policy rate cuts. They get granular in their playbook’s second layer to find countries and sectors they like. Their third layer harnesses mega forces to capture structural shifts within EMs. They see five big structural forces transcending the macro backdrop: digital disruption and artificial intelligence (AI), geopolitical fragmentation, the low-carbon transition, aging populations and the future of finance. They see abundant EM equity opportunities through these m

In [73]:
print(result['answer'])

 The article discusses the BlackRock Investment Institute's new playbook, which leads them to favor emerging market (EM) over developed market (DM) assets. They believe that EM assets have an edge over DM peers in the first layer of their new playbook, the macro assessment. Inflation is cooling enough in key EMs to allow policy rate cuts. They get granular in their playbook’s second layer to find countries and sectors they like. Their third layer harnesses mega forces to capture structural shifts within EMs. They see five big structural forces transcending the macro backdrop: digital disruption and artificial intelligence (AI), geopolitical fragmentation, the low-carbon transition, aging populations and the future of finance. They see abundant EM equity opportunities through these mega forces.<|im_end|>
